This notebook proposes a cheap recipe to train Mistral 7B with DPO. I use the same datasets used by Hugging Face to train Zephyr.

More details in this article: [A Cheap Zephyr 7B Beta: Distilled DPO on Consumer Hardware](https://kaitchup.substack.com/p/a-cheap-zephyr-7b-beta-distilled)

There are two main sections in this notebook: The first one trains SFT and the second one trains DPO. Once SFT is done, I recommend to save your checkpoints somewhere and then to restart the runtime before training DPO.

First, we need all these dependencies:

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

Import all the necessary packages.

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer, DPOTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


# Distilled Supervised Fine-tuning

Load the tokenizer and configure padding

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

Load and preprocess the version of ultrachat prepared by Hugging Face.
Since each row is a full dialog that can be very long, I only kept the first two turns to reduce the sequence length of the training examples.

In [ ]:
def format_ultrachat(ds):
  text = []
  for row in ds:
    if len(row['messages']) > 2:
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content']+"### Human: "+row['messages'][2]['content']+"### Assistant: "+row['messages'][3]['content'])
    else: #not all tialogues have more than one turn
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content'])
  ds = ds.add_column(name="text", column=text)
  return ds
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")

dataset_test_sft = format_ultrachat(dataset_test_sft)
dataset_train_sft = format_ultrachat(dataset_train_sft)


Load the model that we will train with SFT and prepare it for QLoRA.

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define the configuration of LoRA.

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

For this demonstration, I trained for only 300 steps. You should train for at least 3000 steps. One epoch would be ideal.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=2e-5,
        eval_steps=50,
        max_steps=300,
        warmup_steps=30,
        lr_scheduler_type="linear",
)

Start training:

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_train_sft,
        eval_dataset=dataset_test_sft,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/1156 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 207,865
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 300
  Number of trainable parameters = 41,943,040
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded 

Step,Training Loss,Validation Loss
50,1.355100,1.273889
100,1.242400,1.222682
150,1.178900,1.208894


***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-50
tokenizer config file saved in ./results/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-50/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-100
tokenizer config file saved in ./results/checkpoint-100/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-100/special_tokens_m

Step,Training Loss,Validation Loss
50,1.355100,1.273889
100,1.242400,1.222682
150,1.178900,1.208894
200,1.177300,1.200940
250,1.173400,1.196680
300,1.178200,1.195055


***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-200
tokenizer config file saved in ./results/checkpoint-200/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-200/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-250
tokenizer config file saved in ./results/checkpoint-250/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-250/special_token

TrainOutput(global_step=300, training_loss=1.21755916595459, metrics={'train_runtime': 12632.5591, 'train_samples_per_second': 0.38, 'train_steps_per_second': 0.024, 'total_flos': 1.054694248022016e+17, 'train_loss': 1.21755916595459, 'epoch': 0.02})

# Distilled DPO

Load and quantized Mistral 7B that will be trained with DPO.

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load and quantized the reference model that have trained with SFT

In [ ]:

model_ref_name = "mistralai/Mistral-7B-v0.1"
bnb_config_ref = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model_ref = AutoModelForCausalLM.from_pretrained(
          model_ref_name, quantization_config=bnb_config, device_map={"": 0}
)
model_ref = PeftModel.from_pretrained(model_ref, "./results/checkpoint-300/")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Format UltraFeedback for training DPO.

In [ ]:
def format_ultrafeedback(ds):
  text = dict()
  text['chosen'] = []
  text['rejected'] = []
  prompt = []
  for row in ds:
    prompt.append("### Human: "+row['prompt']+"### Assistant: ")
    for col in ['chosen','rejected']:
      text[col].append(row[col][1]['content'])
  ds = ds.rename_column("chosen", "chosen_json")
  ds = ds.rename_column("rejected", "rejected_json")
  ds = ds.rename_column("prompt", "prompt_text")

  ds = ds.add_column(name="chosen", column=text['chosen'])
  ds = ds.add_column(name="rejected", column=text['rejected'])
  ds = ds.add_column(name="prompt", column=prompt)
  return ds
dataset_train_dpo = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
dataset_test_dpo = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="test_prefs[:5%]")

dataset_test_dpo = format_ultrafeedback(dataset_test_dpo)
dataset_train_dpo = format_ultrafeedback(dataset_train_dpo)

Define the configuration of LoRA

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

For this demonstration, I trained for only 100 steps. DPO learns very slowly so you should train for at least 5000 steps. One epoch would be ideal.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results_dpo",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=5e-7,
        eval_steps=50,
        max_steps=200,
        warmup_steps=20,
        lr_scheduler_type="linear",
)

Start DPO training

In [ ]:
trainer = DPOTrainer(
    model,
    model_ref,
    args=training_arguments,
    beta=0.1,
    peft_config=peft_config,
    train_dataset=dataset_train_dpo,
    eval_dataset=dataset_test_dpo,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:255: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the DPOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:262: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:291: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 1
***** Running training *****
  Num examples = 

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
20,0.912900,0.955665,-0.851811,-0.641629,0.415000,-0.210182,-204.748001,-222.435944,-3.054120,-3.045790
40,0.899300,0.952070,-0.837157,-0.631767,0.417500,-0.205389,-204.649399,-222.289398,-3.054138,-3.045726
60,0.908600,0.949276,-0.827149,-0.625535,0.417500,-0.201614,-204.587051,-222.189316,-3.054052,-3.045590
80,0.886800,0.947705,-0.820606,-0.621069,0.420000,-0.199537,-204.542389,-222.123886,-3.054028,-3.045542
100,0.952000,0.947148,-0.818188,-0.619384,0.420000,-0.198804,-204.525543,-222.099731,-3.054044,-3.045543


***** Running Evaluation *****
  Num examples = 400
  Batch size = 1
Saving model checkpoint to ./results_dpo/checkpoint-20
tokenizer config file saved in ./results_dpo/checkpoint-20/tokenizer_config.json
Special tokens file saved in ./results_dpo/checkpoint-20/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 400
  Batch size = 1
Saving model checkpoint to ./results_dpo/checkpoint-40
tokenizer config file saved in ./results_dpo/checkpoint-40/tokenizer_config.json
Special tokens file saved in ./results_dpo/checkpoint-

TrainOutput(global_step=100, training_loss=0.911906623840332, metrics={'train_runtime': 12827.6236, 'train_samples_per_second': 0.249, 'train_steps_per_second': 0.008, 'total_flos': 0.0, 'train_loss': 0.911906623840332, 'epoch': 0.05})